In [1]:
import cv2
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
import os

In [65]:
def cv2_imshow(img):
    image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(image_rgb)
    plt.axis('off')  # Hide axis
    plt.show()

<h1>Get data</h1>

In [ ]:
def split_image(image, rows, cols):
    # Get the dimensions of the image
    height, width = image.shape[:2]
    
    # Calculate the height and width of each sub-image
    sub_height = height // rows
    sub_width = width // cols
    
    # Create a list to store the sub-images
    sub_images = []
    
    for row in range(rows):
        for col in range(cols):
            # Calculate the coordinates of each sub-image
            start_y = row * sub_height
            start_x = col * sub_width
            end_y = (row + 1) * sub_height
            end_x = (col + 1) * sub_width
            
            # Crop the sub-image
            sub_image = image[start_y:end_y, start_x:end_x]
            sub_images.append(sub_image)
    
    return sub_images

def get_center_region(image):
    # Detect the UAV in the frame
    # Get image dimensions
    height, width = image.shape[:2]

    # Calculate the center and radius for the circular crop
    center_x, center_y = width // 2, height // 2
    radius = min(center_x, center_y, width//4)  # Radius is limited by the smallest dimension

    # Crop the rectangular bounding box around the circle
    x1, y1 = center_x - radius, center_y - radius
    x2, y2 = center_x + radius, center_y + radius

    return image[y1:y2, x1:x2]

def extract_images(video_path, output_dir, num_frames):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Get the total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Calculate the interval to extract evenly spaced frames
    interval = total_frames / num_frames
    
    file_name = os.path.join(output_dir, os.path.splitext(os.path.basename(video_path))[0])
    num_images = 0
    for i in tqdm(range(num_frames), desc=f"Progressive {file_name}", leave=True):
        # Set the position of the next frame
        frame_id = round(i * interval)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
        
        # Read the frame
        ret, frame = cap.read()
        if not ret:
            break  # Break if unable to read the frame
        
        center_region = get_center_region(frame)
        cv2.imwrite(f"{file_name}_{i}.jpg", center_region)
        num_images += 1
    
    cap.release()
    return num_images

In [ ]:
# Capture video from the webcam or a video file
video_path = "Images/Sources"
output_path = "Images/Data"
video_names = os.listdir(video_path)
num_imgs = 0
for i, video_name in enumerate(video_names):
    num = extract_images(os.path.join(video_path, video_name), output_path, 400)  # Change to the video file path if needed
    num_imgs += num
print(f"num images: {num_imgs}")

Progressive Images/Data\WIN_20241101_08_28_33_Pro:  28%|██▊       | 110/400 [00:09<00:26, 11.03it/s]


KeyboardInterrupt: 

In [12]:
video_path = "Images/Sources/WIN_20241106_15_10_23_Pro.mp4"
# Open the video file
cap = cv2.VideoCapture(video_path)
    
# Get the total number of frames in the video
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
interval = round(total_frames / 500)
print(interval)

2


<h1>Auto annotation</h1>

In [61]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\Teeho\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [63]:
model = YOLO("yolo11x.pt")  # load a pretrained model (recommended for training)

In [ ]:
# Train the model
results = model.train(data="datasetConfigure.yaml", epochs=100, imgsz=640)

<h1>Train</h1>

<h1>Export</h1>

In [ ]:
# Load the YOLO11 model
model = YOLO("yolo11x_best.pt")

# Export the model to ONNX format
model.export(format="onnx")  # creates 'yolo11n.onnx'

In [ ]:
# Load the exported ONNX model
onnx_model = YOLO("yolo11n.onnx")

# Run inference
results = onnx_model("test.jpg")

<h1>Test</h1>